# Preparação dos dados para modelagem

Na seção anterior realizamos uma serie de analise descritiva dos dados com o intuito de entender a estrutura do dados e identificar as variáveis que podem ser usadas para prever se um cliente vai abandonar o banco ou continuar com ele.

Neste etapa, iremos realizar algumas transformações nos dados para que eles sejam adequados ao modelo preditivo.

Um ponto que deve ser ressaltado, temos dois banco de dados **Abandono_clientes.csv** e **Abandono_teste.csv**, o primeiro foi o banco de dados que usamos na etapa anterior e será este que iremos usar para treinar e testar o modelo. O segundo banco de dados, será utilizado apenas no final para avaliar o desempenho do modelo.

Para construção do nosso modelo utilizaremos o método de [Previsão Conforme](https://medium.com/data-hackers/uma-introdu%C3%A7%C3%A3o-pr%C3%A1tica-%C3%A0-previs%C3%A3o-conforme-de4c7479e021), o objetivo de usar este método em conjunto com outros métodos preditivos é obter um nível de confiança estatística que nos permita avaliar o desempenho do nosso modelo. Desta forma, vamos poder quantificar a incerteza e avaliar o desempenho do nosso modelo.


Nesta fase iremos utilizar uma das principais bibliotecas no cenário de ciência de dados, o [scikit-lerarn](https://scikit-learn.org/) juntamente com os seus algoritmos de machine learning.


In [1]:
# Importando bibliotecas

# Manipulação de dados
import pandas as pd
import numpy as np 

# Visualização
import seaborn as sns 
import matplotlib.pyplot as plt 
from IPython.display import HTML 
import plotly.graph_objects as go 

# Algumas configurações
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Configuração para o notebook e plotagem de imagens
def jupyter_settings():
    %matplotlib inline
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display(HTML('<style>.container { width:100% !important; }</style>'))
    sns.set()

jupyter_settings()


# Machine Learning

# Processamento dos dados
from sklearn.model_selection import train_test_split # Divisão dos dados em treino e teste
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler # Codifica variáveis categóricas em valores numéricos.

# LabelEncoder, OneHotEncoder ->Codifica variáveis categóricas em valores numéricos.
# O primeiro identifica em número inteiro único e não altera a dimensionalidade, mas pode introduzir ordem artificial
# O segunda cada categoria é uma coluna binária e aumenta a dimensionalidade, mas não há ordem artificial
# Normaliza os dados numéricos para facilitar o treinamento dos modelos.
 
from sklearn.compose import ColumnTransformer # Permite aplicar transformações diferentes para diferentes colunas
from sklearn.pipeline import Pipeline # Permite combinar múltiplas etapas do processo de machine learning


# Modelos de Machine Learning:
from sklearn.tree import DecisionTreeClassifier # Importa o classificador baseado em árvores de decisão.
from sklearn.ensemble import RandomForestClassifier # Importa o classificador baseado em florestas aleatórias.
from sklearn.naive_bayes import GaussianNB #  Importa o classificador Naive Bayes Gaussiano.
from sklearn.neighbors import KNeighborsClassifier # Importa o classificador K-Nearest Neighbors (KNN).
from sklearn.svm import SVC # Importa o classificador de Máquinas de Vetores de Suporte (SVM).
from sklearn.neural_network import MLPClassifier # Importa o classificador baseado em redes neurais artificiais (Perceptron Multicamadas).
from sklearn.ensemble import AdaBoostClassifier # Importa o classificador baseado em AdaBoost.
from sklearn.ensemble import GradientBoostingClassifier # Importa o classificador baseado em Gradient Boosting.
from sklearn.ensemble import ExtraTreesClassifier # Importa o classificador baseado em Árvores Extras (Extra Trees).
from xgboost import XGBClassifier #  Importa o classificador baseado no XGBoost, um algoritmo de Gradient Boosting otimizado.
from catboost import CatBoostClassifier # Importa o classificador baseado no CatBoost, especializado em dados categóricos.
from sklearn.linear_model import LogisticRegression # Importa o modelo de Regressão Logística.

# Métricas de desempenho
# Calcula a taxa de recuperação (recall).
# Calcula a curva ROC para análise de desempenho.
# Gera a matriz de confusão.
# Calcula a precisão do modelo
# Calcula a métrica F1, que é a média harmônica entre precisão e recall.
# Calcula a acurácia do modelo.
# Gera um relatório detalhado com precisão, recall e F1-score para cada classe.
from sklearn.metrics import recall_score, roc_curve, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report


Note que iremos remover as seguintes variáveis`RowNumber`', `CustomerId`, `Surname`, pois elas não tem papel nenhum na predição.

In [2]:
df_raw = pd.read_csv('../data/Abandono_clientes.csv', sep=',')
# Vamos fazer uma copia do dataset original

train_transform = df_raw.copy()
train_transform.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True) 
#  

Vamos separar os dados na variável algo $y=$ **Exited** e as variáveis preditoras $X$.

In [3]:
y = train_transform['Exited']

X = train_transform.drop('Exited', axis=1) 

Vamos dividir os dados da seguinte forma:

- 7000 amostras de dados ("`X_train`", "`y_train`") para treinar o modelo;
- 1000 amostras de dados ("`X_test`", "`y_test`") para avaliar o desempenho do modelo em dados "não vistos" no treinamento;
- 1000 amostras de dados ("`X_calib`"`y_calib`") para calibração em dados "não vistos" no treinamento;
- 1000 amostras restantes ("`X_new`"`y_new`") para etapa de previsão conforme e para sua avaliação.

Inicialmente iremos fazer esta distribuição, a depender dos resultados, podemos voltar e alterar estes valores

In [4]:
# Labels are characters but should be integers for sklearn

# Split dos dados em treino e teste
X_train, X_rest1, y_train, y_rest1 = train_test_split(X, y, train_size=7000, random_state=2)

# Dos dados que sobraram, separando em dados de teste
X_test, X_rest2, y_test, y_rest2 = train_test_split(X_rest1, y_rest1, train_size=1000, random_state=42)

# Dos dados que sobraram, separando em calibração e new
X_calib, X_new, y_calib, y_new = train_test_split(X_rest2, y_rest2, train_size=1000, random_state=42)




## Definição de uma baseline

O intuito de criarmos uma baseline é para termos um ponto de partida ao avaliar o desempenho do nosso modelo. Iremos utilizar uma métrica que faz uso de Classificador de taxa aleatória (estimativa ponderada), para ver mais acesse o [artigo](https://towardsdatascience.com/calculating-a-baseline-accuracy-for-a-classification-model-a4b342ceb88f). Desta forma teremos a ZeroR baseline que a taxa mais frequente e a Random Rate (Baseline Ponderada) que é construída a seguir.

In [5]:
# Suponha que `y_train` contém os rótulos da variável alvo
# Cálculo das proporções de cada classe
proporcoes = np.bincount(y_train) / len(y_train)
p_minoritaria = proporcoes[1]  # Proporção da classe Exited = 1
p_majoritaria = proporcoes[0]  # Proporção da classe Exited = 0

# Cálculo das odds. Probabilidade de adivinhar corretamente com base nas proporções.
odds_minoria = p_minoritaria**2
odds_maioria = p_majoritaria**2

# Soma as odds ponderadas para cada classe para obter a precisão esperada de um classificador que adivinha com base nas proporções.
baseline = odds_minoria + odds_maioria

# Exibição dos resultados
print(f"Proporção da Classe Minoritária (Exited = 1): {p_minoritaria:.4f}")
print(f"Proporção da Classe Majoritária (Exited = 0): {p_majoritaria:.4f}")
print()
print(f"Odds de Adivinhar Corretamente a Classe Minoritária: {p_minoritaria}^2 = {odds_minoria:.4f}")
print(f"Odds de Adivinhar Corretamente a Classe Majoritária: {p_majoritaria}^2 = {odds_maioria:.4f}")
print()
print(f"Baseline Ponderado: {baseline:.4f} ou {baseline * 100:.2f}%")

Proporção da Classe Minoritária (Exited = 1): 0.2074
Proporção da Classe Majoritária (Exited = 0): 0.7926

Odds de Adivinhar Corretamente a Classe Minoritária: 0.20742857142857143^2 = 0.0430
Odds de Adivinhar Corretamente a Classe Majoritária: 0.7925714285714286^2 = 0.6282

Baseline Ponderado: 0.6712 ou 67.12%


Note que nossa Baseline Ponderada é de $67,12\%$ e a ZeroR Baseline é de $79,26\%$ estes valores serão os valores de avaliação inicial.

## Transformação dos dados

Vamos fazer a separação das variáveis numéricas e categóricas. Decidi adicionar `HasCrCard`, `IsActiveMember` como variáveis categóricas, mesmo sendo números. No futuro posso voltar aqui e rever esta decisão. 

As transformações que vamos fazer são:

- **StandardScaler** para padronizar os valores, inicialmente não irei tratar os outliers, desejo ver como os modelos atual considerando esses valores, também não irei o min-max scaler para os valores que não possuem outliers.
- **OneHotEncoder** para transformar as variáveis categóricas do tipo `Gender` e `Geography` e valores binários. 

In [7]:
# Variáveis numéricas categóricas
categorical_numeric_features = ['HasCrCard', 'IsActiveMember', 'NumOfProducts']

# Variáveis numéricas contínuas
numeric_features = [col for col in X_train.select_dtypes(include=np.number).columns if col not in categorical_numeric_features]

# Variáveis categóricas
categorical_features = X_train.select_dtypes(include=['object']).columns.tolist() + categorical_numeric_features

# Transformadores
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Pré-processador
preprocesso = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Criação do pipeline para conversão
preprocesso = ColumnTransformer(transformers= [('num', numeric_transformer, numeric_features),
                                            ('cat', categorical_transformer, categorical_features)])


## Modelagem 

Vamos criar uma lista com os seguintes modelos




In [14]:
# definindo uma lista de modelos
list_models = [('LOGISTIC REGRESSION', LogisticRegression(solver='lbfgs')),
               ('DECISION TREE', DecisionTreeClassifier()),
               ('KNN', KNeighborsClassifier()),
               ('SVC', SVC()),
               ('RANDOM FOREST', RandomForestClassifier(n_estimators=1000)),
               ('XGB', XGBClassifier(n_estimators=300, eval_metric='logloss', seed=42)),
               ('ABC', AdaBoostClassifier(n_estimators=50, learning_rate=1, random_state=42)),
               ('ExT', ExtraTreesClassifier()),
               ('GBC', GradientBoostingClassifier()),
               ('CatBC', CatBoostClassifier(verbose=0))
               ]

In [19]:
for name, model in list_models:
    clf = Pipeline(steps=[('preprocesso', preprocesso),
                          ('classificação', model)])
    
    clf.fit(X_train, y_train)

    train_score = clf.score(X_train, y_train)
    test_score = clf.score(X_test, y_test )

    print(name)
    print('-'*54)
    print('Treino', '\t', train_score)
    print('Validação', test_score, '\n')
    print(classification_report(y_test, clf.predict(X_test)))
    print('='*54, '\n')

LOGISTIC REGRESSION
------------------------------------------------------
Treino 	 0.842
Validação 0.846 

              precision    recall  f1-score   support

           0       0.86      0.96      0.91       793
           1       0.73      0.40      0.52       207

    accuracy                           0.85      1000
   macro avg       0.80      0.68      0.71      1000
weighted avg       0.83      0.85      0.83      1000


DECISION TREE
------------------------------------------------------
Treino 	 1.0
Validação 0.816 

              precision    recall  f1-score   support

           0       0.89      0.88      0.88       793
           1       0.55      0.57      0.56       207

    accuracy                           0.82      1000
   macro avg       0.72      0.73      0.72      1000
weighted avg       0.82      0.82      0.82      1000


KNN
------------------------------------------------------
Treino 	 0.8705714285714286
Validação 0.842 

              precision    reca

In [ ]:
# Modelo 
model = DecisionTreeClassifier().fit(X_train, y_train)
# Check accuracy
y_pred = model.predict(X_test)
print("Accuracy:", (y_pred == y_test).mean())

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred)
print(pd.DataFrame(cm,  index=['Não Saiu', 'Saiu'],      # Rótulos das classes verdadeiras
    columns=['Não Saiu', 'Saiu']))

Accuracy: 0.806
          Não Saiu  Saiu
Não Saiu       696    97
Saiu            97   110


In [62]:

#Get the "probabilities" from the model
predictions = model.predict_proba(X_calib)

# Get for each instance the highest probability
high_prob_predictions = np.amax(predictions, axis=1)

# Select the predictions where probability over 99%
high_p_Exited = np.where(high_prob_predictions >= 0.95)

# Let's count how often we hit the right label
its_a_match = (model.predict(X_calib) == y_calib)
coverage = np.mean(its_a_match.values[high_p_Exited])
print(round(coverage, 3))

0.781


In [28]:
categorical_features

['Geography', 'Gender']

In [27]:
numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# 2. Definir transformações
numeric_transformer = StandardScaler()  # Padronizar variáveis numéricas
categorical_transformer = OneHotEncoder(handle_unknown='ignore')  # Codificar variáveis categóricas

# 3. Criar o pré-processador
preprocesso = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 4. Definir o classificador com parâmetros recomendados
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

# 5. Criar o pipeline completo
pipeline = Pipeline(steps=[
    ('preprocesso', preprocesso),
    ('classificação', xgb_model)
])

# 6. Treinar o pipeline
pipeline.fit(X_train, y_train)

# 7. Avaliar o pipeline
train_score = pipeline.score(X_train, y_train)
test_score = pipeline.score(X_test, y_test)

print(f"Acurácia no treino: {train_score:.2f}")
print(f"Acurácia no teste: {test_score:.2f}")

/home/mhpedro/miniconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [23:12:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


AttributeError: 'super' object has no attribute '__sklearn_tags__'